# Mount the drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/MyDrive/RGBcode")

# Traing

## Framework A
File path needs to be changed for the corresponding one.

In [ ]:
!python train_A.py \
    --model_type compression_gan \
    --name stage_A\
    --regime low 


## Use tensorboard to see the loss

In [ ]:
%load_ext tensorboard
%tensorboard --logdir \
/content/gdrive/MyDrive/RGBcode/experiments/stage_A/tensorboard/train/

# Testing
File path needs to be changed for the corresponding one.

In [ ]:
## FAH
!python3 "/content/gdrive/MyDrive/RGBcode/compress2.py" \
--metrics --reconstruct \
--ckpt_path /content/gdrive/MyDrive/RGBcode/experiments/FAH/checkpoints/FA_high_epoch235_idx24910.pt \
--decoder /content/gdrive/MyDrive/RGBcode/ckpt/FAH/checkpoint_A_high/model_epoch_235.pth \
--output_dir /content/gdrive/MyDrive/RGBcode/MMSE/FAH_1 

# Calculate MSE

In [ ]:
from PIL import Image
import numpy as np
from sklearn.metrics import mean_squared_error
from skimage.metrics import peak_signal_noise_ratio

def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

def save_image( npdata, outfilename ) :
    img = Image.fromarray( np.asarray( np.clip(npdata,0,255), dtype="uint8"), "L" )
    img.save( outfilename )

In [ ]:
# alpha_0
# p = [0, 0.1, 0.25, 0.3, 0.5, 0.6, 0.75, 0.8, 0.9]
p = [0.005, 0.05, 0.01, 0.1, 0.2, 0.4]

mse = [float("inf")] * len(p)
for j in range(len(p)):
  for i in range(1, 25):

    ### Kodak
    src_p = "/content/gdrive/MyDrive/RGBcode/data/coco2014/test/" + "kodim" + str(i).zfill(2) + ".png"
    # tar_p = "/content/gdrive/MyDrive/RGBcode/data/Kodak_compression_" + str(p[j]) + "/kodim" + str(i).zfill(2) + ".png"
    tar_p = "/content/gdrive/MyDrive/RGBcode/data/L" + str(p[j]) + "/kodim" + str(i).zfill(2) + ".png"

    ### Set 14
    # src_p = "/content/gdrive/MyDrive/RGBcode/data/set14/" + "img_"  + str(i).zfill(3) + " .png"
    # tar_p = "/content/gdrive/MyDrive/RGBcode/data/set14_" + str(p[j]) + "/img_"  + str(i).zfill(3) + " .png"
    # tar_p = "/content/gdrive/MyDrive/RGBcode/data/s14_fa_" + str(p[j]) + "/img_"  + str(i).zfill(3) + " .png"

    ### G10
    # src_p = "/content/gdrive/MyDrive/RGBcode/data/G10" + "/im_"  + str(i) + ".bmp"
    # tar_p = "/content/gdrive/MyDrive/RGBcode/data/G_" + str(p[j]) + "/im_"  + str(i) + ".png"
    # tar_p = "/content/gdrive/MyDrive/RGBcode/data/G10_fa_" + str(p[j]) + "/im_"  + str(i) + ".png"

    s = load_image(src_p)
    t = load_image(tar_p)

    if len(s.shape) == 2:
      continue

    s = s.reshape(-1, s.shape[0]*s.shape[1]*s.shape[2])
    t = t.reshape(-1, t.shape[0]*t.shape[1]*t.shape[2])

    mse[j] = min(mse[j], mean_squared_error(s,t))
  
  mse[j] = round(np.sqrt(mse[j]), 3)